In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/')

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
import os

In [ ]:
# Defining meta-information for model
IMG_WIDTH = 128
IMG_HEIGHT = 128
interpolation_method = 'lanczos' # Reasoning: keeps finer details in downsampling | didn't really do much for how well it predicts
#interpolation_method = 'nearest' # Reasoning: keeps finer details in downsampling

# Defining Paths
dirname = os.path.dirname(__file__)
base_path = os.path.join(dirname, 'correct_path_to_dataset/1_classification/1_classification/') # define the correct path
train_path = base_path + 'Train/'
test_path = base_path + 'Test/'

train_csv = train_path + 'labels_train.csv'
test_csv = test_path + 'labels_test.csv'

# Read CSV files
train_labels = pd.read_csv(train_csv)
test_labels = pd.read_csv(test_csv)

# Define image data generators for train and test sets
#train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split=0.2,brightness_range=[1.0,1.5])
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split=0.2)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Get images from directories using data generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(IMG_WIDTH, IMG_HEIGHT),  # Set your image dimensions
    batch_size=32,
    class_mode='binary',
    subset='training',
    #interpolation=interpolation_method,
    shuffle=True)  # For binary classification

validation_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(IMG_WIDTH, IMG_HEIGHT),  # Set your image dimensions
    batch_size=32,
    class_mode='binary',
    subset='validation',
    #interpolation=interpolation_method,
    shuffle=True)  # For binary classification

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(IMG_WIDTH, IMG_HEIGHT),  # Set your image dimensions
    batch_size=32,
    #interpolation=interpolation_method,
    class_mode='binary')  # For binary classification

In [ ]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
# Defining Model
model = models.Sequential()
model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3), kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l1_l2(0.01)))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l1(0.005)))
model.add(layers.MaxPooling2D((4, 4)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
# Show Model Summary
model.summary()

In [ ]:
# Why adam for optimizer? Generally good performance according to https://stackoverflow.com/questions/37214884/how-do-i-choose-an-optimizer-for-my-tensorflow-model and common use in practice according to lecture slides
# Why binary_crossentropy for loss? We work with binary classification on a balanced dataset (labels 1 and 0 occur each 400 and 100 times in the train and test datasets respectively).
# Why accuracy for metrics? Again, balanced dataset so accuracy is appropriate
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.BinaryIoU(target_class_ids=[0], threshold=0.5)]) # 0 for some reason true labels
fitting_information = model.fit(train_generator, epochs=8,validation_data=validation_generator)

In [ ]:
# Test model and print validation and test accuracies
test_loss, test_acc, test_binary_io_u = model.evaluate(test_generator, verbose=2)

print(f"training loss: {fitting_information.history['loss'][-1]}, training accuracy: {fitting_information.history['accuracy'][-1]}, training binary_io_u: {fitting_information.history['binary_io_u_3'][-1]}")
print(f"validation loss: {fitting_information.history['val_loss'][-1]}, validation accuracy: {fitting_information.history['val_accuracy'][-1]}, validation binary_io_u: {fitting_information.history['val_binary_io_u_3'][-1]}")
print(f"test loss: {test_loss}, test accuracy: {test_acc}, test binary_io_u: {test_binary_io_u}")

In [ ]:
# Save Model
model_save_path = os.path.join(dirname,'models/ipda_task_1_clean_18')
model.save(model_save_path)